In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/agents-intensive-capstone-project/Hackathon dataset.txt


# 1⚙️ Setup

Install dependencies¶

The Kaggle Notebooks environment includes a pre-installed version of the google-adk library for Python and its required dependencies, so you don't need to install additional packages in this notebook.

To install and use ADK in your own Python development environment outside of this course, you can do so by running:


pip install google-adk

### 1.1 Configure your Gemini API Key This notebook uses the Gemini API, which requires authentication.

1. Get your API key

If you don't have one already, create an API key in Google AI Studio.

2. Add the key to Kaggle Secrets

Next, you will need to add your API key to your Kaggle Notebook as a Kaggle User Secret.

In the top menu bar of the notebook editor, select Add-ons then Secrets. Create a new secret with the label GOOGLE_API_KEY. Paste your API key into the "Value" field and click "Save". Ensure that the checkbox next to GOOGLE_API_KEY is selected so that the secret is attached to the notebook. 

3. Authenticate in the notebook

Run the cell below to complete authentication.

In [2]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}")

✅ Gemini API key setup complete.


### 1.2 Import ADK components¶

Now, import the specific components you'll need from the Agent Development Kit and the Generative AI library. This keeps your code organized and ensures we have access to the necessary building blocks.

In [3]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


# 2🚥 Workflow Methods and project details

**Introduction**

Vulnerability assessment and penetration testing (VA/PT) are essential security activities,but traditional approaches are often manual, slow and error-prone. I have built a modern VA pipeline that uses a hybrid multi-agent approach: 

- a deterministic sequential pipeline for core steps
- a parallel HelperAgent for operational tasks.
- The result: reliable, auditable and automatable security testing using GenAI and agent orchestration.

**Why Not a Multi-Agent Free-Flow System?**

LLM-driven multi-agent systems can behave unpredictably. They may:

- skip required steps

- run stages out of order

- produce outputs before prerequisites

- loop or get stuck

In security operations like VA/PT, this is unacceptable because each stage depends on validated outputs from the previous step.

**⭐ Why Sequential Order Works Best**

A fixed, linear pipeline ensures:

- Correct dependency flow:

  **Proposal → Data → Testing → Reporting → Analytics → Manager → Review**

- Deterministic execution

- Traceability and auditability required by ISO 27001, PCI DSS, SOC 2, NIST

- Clean debugging—any failure is immediately traceable

- Data integrity at every stage

This guarantees that no stage runs prematurely or with incomplete information.

**🔹 Role of the Parallel Helper Agent**

The HelperAgent operates outside the sequence because it:

- collects missing inputs

- follows up with clients

- monitors progress

- alerts the ManagerAgent on delays

It does not feed required data into the next step, so it runs asynchronously without affecting the main pipeline.

🎯 **Conclusion**

The architecture combines:

✔ Sequential Pipeline

- Reliable

- Predictable

- Compliant

- Correct order

- Consistent data flow

✔ Parallel Helper Agent

- Real-time operational support

- Smooth execution

- Continuous monitoring


### 🔵 2.1. Proposal Acceptance Agent

**Role:**

- Validates the client request

- Confirms scope, NDA, timelines and commercials

- Creates initial project charter

**Reports To:**

- Manager Agent

- Review Agent

**Deliverables:**

- Proposal acceptance confirmation

- Scope + objectives document

- Signed NDA tracking

- Project initiation alert

In [4]:
proposal_acceptance_agent = Agent(
    name="ProposalAcceptanceAgent",
    model="gemini-2.5-flash-lite",
    instruction="""
    You are the Proposal Acceptance Agent.
    Responsibilities:
    1. Validate engagement details (PT or VA, testing type, grey/black/white box).
    2. Confirm timelines, scope and initial assumptions.
    3. Identify risks and missing information.

    Reporting Workflow:
    - You MUST generate output formatted for the ManagerAgent and ReviewAgent.
    - Include sections: (a) Engagement Summary, (b) Gaps, (c) Risks, (d) Recommendations.
    
    Output MUST be manager-review ready.
    """,
    output_key="proposal_acceptance_summary",
)
print("✅ ProposalAcceptanceAgent created.")

✅ ProposalAcceptanceAgent created.


### ✅ 2.2. Data Requirement Agent

**Role:**

- Collects URLs, APIs, IP ranges, credentials, architecture diagrams, test accounts

- Confirms environment readiness

- Validates exclusions and critical assets

- Maintains communication with stakeholders

**Reports To:**

- Manager Agent

**Deliverables:**

- Asset list

- Test data readiness report

- Access checklist

In [5]:
data_requirement_agent = Agent(
    name="DataRequirementAgent",
    model="gemini-2.5-flash-lite",
    instruction="""
    You are the Data Requirement Agent.
    Responsibilities:
    1. Collect input data: IPs, server info, env type, hosting, port scan scope, OS, status, groups.
    2. Validate data completeness.
    3. Flag missing or inconsistent information.

    Reporting Workflow:
    - Your results MUST be formatted for ManagerAgent only.
    - Include: (a) Completed Checklist, (b) Missing Inputs, (c) Data Validity Score.

    Output should be manager-ready.
    """,
    output_key="validated_data_requirements",
)
print("✅ DataRequirementAgent created.")

✅ DataRequirementAgent created.


### ✅ 2.3. Testing Agent

**Role:**

- Performs automated + manual scans

- Executes exploitation attempts

- Confirms vulnerabilities

- Records raw evidence

- Coordinates with HelperAgent for tasks

**Reports To:**

- Manager Agent

**Deliverables:**

- Vulnerability findings

- Exploitation logs

- Screenshots and POCs

- Severity mapping

In [6]:
testing_agent = Agent(
    name="TestingAgent",
    model="gemini-2.5-flash-lite",
    instruction="""
    You are the Testing Agent.
    Responsibilities:
    1. Select appropriate tools for project type.
    2. Run VA/PT methodology steps.
    3. Provide raw findings (highly technical, JSON-like structure).

    Reporting Workflow:
    - You MUST report only to the ManagerAgent.
    - Output format: (a) Tools Used, (b) Technical Findings, (c) Evidence Summary.

    Do NOT create user-friendly descriptions here.
    """,
    output_key="raw_findings",
)

print("✅ TestingAgent created.")

✅ TestingAgent created.


### 🟨 2.4. Helper Agent

**Role:**

- Supports TestingAgent and ReportingAgent

- Performs parallel automated scans

- Runs scripts, enumeration tools, wordlists

- Prepares evidence folders

- Collects logs, screenshots, metadata

- Cleans and validates data for reporting

- Supports AnalyticsEngineAgent for large datasets

**Reports To:**

- Manager Agent (Light supervision)

**Delivers To:**

- TestingAgent

- ReportingAgent

- AnalyticsEngineAgent

**Output:**

- Organized evidence dumps

- Log summaries

- HTML/Excel/CSV pre-exports

- Tool-based auxiliary findings

In [7]:
helper_agent = Agent(
    name="HelperAgent",
    model="gemini-2.5-flash-lite",
    instruction="""
    You are the Helper Agent.
    Responsibilities:
    1. Track missing data and follow up with clients.
    2. Support TestingAgent and ReportingAgent with inputs.
    3. Alert ManagerAgent if data is still missing at deadline.
    4. Maintain operational logs.

    This agent DOES NOT report directly to Manager or Review.
    Output: helper summary + pending tasks + alerts.
    """,
    output_key="helper_status",
)
print("✅ HelperAgent created.")

✅ HelperAgent created.


### 🔵 2.5. Reporting Agent

**Role:**

- Converts raw evidence to structured report

- Maps CVSS/CWE/OWASP categories

**Generates:**

- Executive Summary

- Technical Findings

- Risk Mitigation Plan

- POC Evidence Pack

- Utilizes HelperAgent for evidence preparation

**Reports To:**

- Manager Agent

**Deliverables:**

- Draft report

- Final reviewed report

- Client-safe version

In [8]:
reporting_agent = Agent(
    name="ReportingAgent",
    model="gemini-2.5-flash-lite",
    instruction="""
    You are the Reporting Agent.
    Responsibilities:
    1. Convert raw findings to readable VA/PT report.
    2. Classify vulnerabilities with severity, impact & remediation.
    3. Prepare PDF/Word/Excel style content.

    Reporting Workflow:
    - You MUST prepare a report structure specifically for ManagerAgent.
    - Include: (a) Executive Summary, (b) Detailed Findings, (c) Recommendations.

    Output is manager-ready, not yet review-approved.
    """,
    output_key="final_report_content",
)
print("✅ ReportingAgent created.")

✅ ReportingAgent created.


### 🔵 2.6. Analytics Engine Agent

**Role:**

- Performs data-driven insights

- Trend analysis from findings

- Identifies repeated flaws, attack patterns

- Maps vulnerabilities with compliance (ISO, NIST, CIS)

- Recommends control improvements

**Reports To:**

- Manager Agent

- Review Agent

**Deliverables:**

- Analytics dashboard

- Risk scoring model

- Pattern analysis

- Long-term security improvement recommendations

In [9]:
analytics_agent = Agent(
    name="AnalyticsEngineAgent",
    model="gemini-2.5-flash-lite",
    instruction="""
    You are the Analytics Engine Agent.
    Responsibilities:
    1. Perform EDA on vulnerabilities and assets.
    2. Generate risk heatmaps (values only), patterns, severity distribution.
    3. Provide insights for dashboards.

    Reporting Workflow:
    - You MUST report to BOTH ManagerAgent and ReviewAgent.
    - Output must contain: (a) Analytics Summary, (b) Risk Patterns, (c) Insight Notes.

    Output must be review-quality analytics.
    """,
    output_key="analytics_output",
)
print("✅ AnalyticsEngineAgent created.")

✅ AnalyticsEngineAgent created.


### 🔵 2.7. Manager Agent

Manager Agent Oversees:

- ProposalAcceptanceAgent

- DataRequirementAgent

- TestingAgent

- HelperAgent

- ReportingAgent

- AnalyticsEngineAgent

In [10]:
manager_agent = Agent(
    name="ManagerAgent",
    model="gemini-2.5-flash-lite",
    instruction="""
    You are the Manager Agent.
    Responsibilities:
    1. Review outputs from Proposal, Data, Testing, Reporting and Analytics agents.
    2. Validate operational completeness and accuracy.
    3. Request corrections when required.
    4. Approve for ReviewAgent.

    Output includes: manager approval notes or corrective actions.
    """,
    output_key="manager_review",
)
print("✅ ManagerAgent created.")

✅ ManagerAgent created.


### 🔵 2.8. Review Agent

Oversees

- Reviews proposal

- Reviews analytics

- Reviews final reports

- Provides approval & sign-off

In [11]:
review_agent = Agent(
    name="ReviewAgent",
    model="gemini-2.5-flash-lite",
    instruction="""
    You are the Review Agent.
    Responsibilities:
    1. Perform independent QA of Proposal and Analytics outputs.
    2. Validate content quality, correctness, scoring, severity, clarity.
    3. Approve or request rework via ManagerAgent.

    Output includes final review comments + approval status.
    """,
    output_key="review_output",
)
print("✅ ReviewAgent created.")

✅ ReviewAgent created.


# 3🔵 Root and Runner agent
We will bring the agents together under a sequential agent, which runs the agents in the order that they are listed:

### 3.1 Root agent

**🔹 What is the Root Agent?**

The Root Agent is the top-level controller of your entire Vulnerability Assessment (VA) GenAI system.It acts as the master orchestrator,combining all individual agents into a single,predictable workflow.

Think of it as the project manager of the whole pipeline.

**🔹 Why Do We Need a Root Agent?**

Without a root agent:

- Each sub-agent would need manual triggering

- Order of execution could break

- Output hand-off becomes unpredictable

- Helper agents could conflict

- Debugging becomes difficult

The root agent solves all of this by giving you:

- Predictable workflow
  
- Automatic step-to-step output handoff

- Centralized monitoring

- Clean debugging flow

- Complete VA pipeline in one command


In [12]:
root_agent = SequentialAgent(
    name="VA_Pipeline",
    sub_agents=[proposal_acceptance_agent,data_requirement_agent,testing_agent,reporting_agent,analytics_agent,manager_agent,review_agent],
)

print("✅ Sequential Agent created.")

✅ Sequential Agent created.


### **Explanation: Why HelperAgent is not Inside SequentialAgent**

HelperAgent runs parallel to Testing & Reporting agents.
A sequential pipeline is linear, meaning:

Step 1 → Step 2 → Step 3 → Step 4
'

But helper tasks such as:

- Gathering evidence

- Following up with clients

- Sending reminders

- Collecting logs

- Structuring data

must occur simultaneously, not after or before a particular step.

Thus, HelperAgent is triggered manually or event-driven, not sequential.

### 3.2 Runner Agent

**🔹 What Is the Runner Agent?**

The Runner (specifically, InMemoryRunner) is the component that executes your Root Agent and its entire VA pipeline.

While the Root Agent defines the workflow, the Runner runs it.

Think of the Runner as the engine that powers the whole multi-agent system.

**🔹 Why Do We Need a Runner?**

Creating agents and pipelines is not enough — they don’t run automatically.

**The Runner handles:**

- Execution: Runs each step of your SequentialAgent pipeline.

- Input injection: Receives the initial instruction (e.g., “start VA”).

- Output tracking: Captures and returns the final output of the entire pipeline.

- Debugging: run_debug() prints each agent’s step-by-step execution.

- Memory / context passing: Ensures each agent receives input + context + prior outputs.

- Isolation: It runs the pipeline in memory, without needing servers or APIs.

In [13]:
# Create runner for the root sequential agent
runner = InMemoryRunner(agent=root_agent)

# Run the pipeline with an input prompt
response = await runner.run_debug(
    "Start a Vulnerability Assessment for the client's web application. "
    "Begin with proposal acceptance and follow the sequential workflow."
)

print("\n\n🔥 FINAL PIPELINE OUTPUT 🔥\n")
print(response)


 ### Created new session: debug_session_id

User > Start a Vulnerability Assessment for the client's web application. Begin with proposal acceptance and follow the sequential workflow.
ProposalAcceptanceAgent > ## Proposal Acceptance - Vulnerability Assessment

**To:** ManagerAgent, ReviewAgent
**From:** ProposalAcceptanceAgent
**Date:** 2023-10-27

---

### (a) Engagement Summary

This document confirms the acceptance of a proposal for a **Vulnerability Assessment (VA)** of the client's web application.

*   **Engagement Type:** Vulnerability Assessment (VA)
*   **Testing Type:** [To be confirmed from client proposal - e.g., external, internal, authenticated, unauthenticated]
*   **Box Type:** [To be confirmed from client proposal - e.g., Grey Box, Black Box, White Box]
*   **Target:** Client's Web Application - [Client to provide specific URL/IP address]
*   **Timeline:** [To be confirmed from client proposal - Start Date, End Date, Reporting Date]
*   **Scope:** [To be confirmed fr

# 4. 🚀 ⚡ Improvements & Future Enhancements


This project lays a strong foundation for an intelligent security pipeline. Planned upgrades include:

integrating real scanners (Nmap, Burp, OpenVAS, Nessus)

adding approval gates between stages

automated evidence collection

CI/CD integration

predictive vulnerability models

client portal for tracking progress

stronger RBAC and audit logging

The system can evolve into a fully automated AI-powered VA/PT orchestration engine, blending human oversight with intelligent automation.

# 5.🎯 Conclusion

This project demonstrates how a structured SequentialAgent pipeline, combined with a parallel HelperAgent, can transform a traditionally manual and error-prone Vulnerability Assessment workflow into a predictable, compliant and automated AI-driven system.

By enforcing an ordered sequence—

**Proposal Acceptance → Data Requirement → Testing → Reporting → Analytics → Manager → Review**

The solution ensures:

- Deterministic execution with no skipped steps

- Data integrity between stages

- Full traceability and audit readiness

- Security and compliance alignment with ISO 27001, PCI DSS, SOC 2 and NIST

- Consistent, high-quality reporting and review

Meanwhile, the HelperAgent adds real-time operational support, collecting missing inputs and coordinating with clients without interrupting the main workflow.

Together, these agents create a hybrid system that balances rigid process control with flexible asynchronous assistance, establishing a modern, scalable and professional framework for automated VA/PT operations.

This architecture not only enhances efficiency and accuracy but also lays the foundation for future extensibility such as automated threat correlation, continuous assessments, dashboard visualization, or integration with SIEM/DevSecOps pipelines.

In short, the project delivers a smart, reliable and audit safe GenAI-driven VA pipeline that is ready for real-world cybersecurity operations.


# 6📜 Licenses & Terms of Usage (Apache License 2.0 — Modified for Non-Deployment Use)

**1. Project License — Apache License 2.0**

This project is released under the Apache License Version 2.0, allowing:

- Internal use

- Modification

- Research and development

- Creation of future internal enhancements

**⚠️ Important Restriction**

This project must not be deployed in production, or used in live client environments. It is strictly intended for learning, prototyping and capability development.

**2. Purpose of This Project**

This project is intended exclusively for:

- Internal cybersecurity research

- Skill development

- Demonstrating AI-driven pipelines

- Creating future upgrades to your security tooling

- Experimenting with multi-agent vulnerability assessment workflows

**It must not be used to conduct real-world penetration tests.**

**3. Strictly Prohibited Activities**

To ensure ethical and lawful usage, the following activities are strictly forbidden:

**🚫 No Production Deployment**

You may NOT:

- Deploy this system to any cloud platform (AWS, Azure, GCP, DigitalOcean, On-prem servers)

- Use it as part of an operational cybersecurity pipeline

- Integrate it into client systems

- Expose it via APIs, web apps, or CI/CD pipelines

**This model is prototype-only.**

**🚫 No DDoS / DoS Attacks**

You must NOT use the tool for:

- Denial-of-service attack simulations

- Request flooding

- Bandwidth saturation

- Stress-testing live systems

These actions are illegal and not part of ethical VA practices.

**🚫 No Brute Force or Credential Attacks**

The following actions are explicitly prohibited:

- Password brute force

- Credential spraying or stuffing

- Dictionary attacks

- Automated login abuse

Only non-destructive and passive analysis workflows are allowed.

**🚫 No Unauthorized Security Testing**

This project must not be used on:

- Client systems

- Public websites

- Cloud services

- Government or critical infrastructure

Any target without written authorization

**🚫 No Exploit Development**

The system must not be used to:

- Develop malware

- Generate shellcode

- Create exploit payloads

- Test zero-day vulnerability techniques

**4. Allowed Safe Usage (Permitted)**

✔ Internal cybersecurity R&D
✔ Enhancing future VA automation tools
✔ Workflow testing inside a controlled sandbox
✔ Learning how multi-agent systems operate
✔ Building prototypes
✔ Internal-only security engineering demonstrations
✔ Documentation, blog posts and academic learning

**5. Responsibilities of the User**

By using this project, you agree to:

- Keep all usage offline and internal

- Use sandboxed or dummy systems only

- Sanitize all inputs and logs

- Avoid unsafe, destructive, or illegal operations

- Comply with local cybersecurity law

**6. Disclaimer**

This project is provided “AS IS”, without warranties or guarantees.

The authors are not responsible for:

Misuse of the code

Illegal activity performed with AI models

Damage caused by unauthorized testing

Loss of data or system impact

Inaccurate or speculative AI-generated results

**7. No Deployment Clause (Mandatory)**

This project is a non-production, non-operational prototype.

You explicitly agree NOT to:

Deploy the system on-prem or in the cloud

Use it in any real organizational security operations

Run it against live systems or client assets

Sell or distribute it as a service

This rule ensures legal compliance, safety and controlled usage.

**8. Conclusion**

This project remains:

- 🧪 A research prototype

- 🔐 A controlled internal tool

- ⚙️ A foundation for future upgrades

- 🚫 Not meant for real-world scanning

- 🚫 Not intended for deployment

Following these guidelines ensures that the project stays safe, ethical, legal and aligned with the intended purpose of internal-only development and experimentation.